In [ ]:
import random
import operator


class Submission:
    """
    Uses the function that predicts the users interests for an item to find the top 25 ratings and writes it to a file in
    the format specified by the RecSys challenge.
    """
            
    #  Dummy function returning the predicted rating for a user-item pair.
    def predicted_rating(self, user, item):
        return random.random()  # Todo: Implement this method.

    # Given a user and a list of items, predict the user's rating for this items and return a sorted list of the items
    # from highest predicted ratings to lowest predicted rating.
    def predict_ratings(self, user, items):

        # Generates a dictionary with the items as keys and the predicted rating for the user to the items as the values
        predicted_item_ratings = {}
        for item in items:
            predicted_rating = self.predicted_rating(user, item)
            predicted_item_ratings[item] = predicted_rating

        # Sort the accommodations by highest predicted rating.
        sorted_list_of_tuples = sorted(predicted_item_ratings.items(), key=operator.itemgetter(1), reverse=True)
        items_sorted = [tuple[0] for tuple in sorted_list_of_tuples]

        return items_sorted
